In [ ]:
%pip install plotly
%pip install nbformat
%pip install ipython
%pip install numpy

In [ ]:
from __future__ import annotations
import plotly.graph_objects as go
import numpy as np
from numpy.typing import NDArray
from numpy import float64, floating
from typing import Any, Optional

# Analog'tan analog'a çevrim (modülasyon)
Analog sinyal, zaman komponentine bağlı olarak değişen sürekli (continuous) bir özelliğe sahip olan sinyaldir.

## Modülasyonun hedefi
Analogtan analog'a çevrim işlemi ile analog veri, analog bir sinyal haline getirilir. 

Analog bir sinyalin bulunduğu durumda ise hedeflenen farklı bir *taşıyıcı sinyal*den yararlanılarak bilgiyi taşıyan analog sinyali aktarmaktır.

## Modülasyon neden gerekir
- Analog (bilgi) sinyalinin iletileceği ortam, eldeki sinyalin frekansı için uygun olmadığı zaman
- Sinyal iletimi [bandpass](https://en.wikipedia.org/wiki/Band-pass_filter) bir kanaldan yapılacağı zaman
- Sinyal iletimi için kullanılacak ortamda (kanalda) birden fazla sinyal bulunması durumunda

In [ ]:
class BasicSignal:
    _frequency: int
    _amplitude: int
    _phase: int
    _sampling_frequency: int
    _length: int
    _signal_w: NDArray[floating]
    _signal_t: NDArray[floating]
    
    def __init__(self, frequency: int, amplitude: int, phase: int, sampling_frequency: int, length: int):
        self._frequency = frequency
        self._amplitude = amplitude
        self._phase = phase
        self._sampling_frequency = sampling_frequency
        self._length = length
        self._compute_signal()
        
    def _compute_signal(self) -> None:
        self._signal_t = np.linspace(0, self._length, self._sampling_frequency * self._length)
        # self._signal_t = np.arange(0, self._length, 1 / self._sampling_frequency)
        self._signal_w = self._amplitude * np.sin(2 * np.pi * self._frequency * self._signal_t + self._phase)
    
    def get_signal_elements(self):
        return self._signal_w, self._signal_t
    
    @classmethod
    def modify_signal(cls, signal: 'BasicSignal', frequency: Optional[int] = None, amplitude: Optional[int] = None, phase: Optional[int] = None, sampling_frequency: Optional[int] = None, length: Optional[int] = None) -> None:
        if frequency is not None:
            signal._frequency = frequency
        if amplitude is not None:
            signal._amplitude = amplitude
        if phase is not None:
            signal._phase = phase
        if sampling_frequency is not None:
            signal._sampling_frequency = sampling_frequency
        if length is not None:
            signal._length = length
        signal._compute_signal()
    
    def plot_signal(self, name: str = 'Unnamed signal') -> None:
        fig = go.Figure(layout=go.Layout(title=name, xaxis=dict(title='t-component'), yaxis=dict(title='w-component')))
        fig.add_trace(go.Scatter(x=self._signal_t, y=self._signal_w))
        fig.show()

    @classmethod
    def copy_and_modify(cls, signal: 'BasicSignal', frequency: Optional[int] = None, amplitude: Optional[int] = None, phase: Optional[int] = None, sampling_frequency: Optional[int] = None, length: Optional[int] = None) -> 'BasicSignal':
        signal_copy: 'BasicSignal' = cls(signal._frequency, signal._amplitude, signal._phase, signal._sampling_frequency, signal._length)
        cls.modify_signal(signal_copy, frequency, amplitude, phase, sampling_frequency, length)
        return signal_copy
    
    @classmethod
    def from_tuple(cls, signal: tuple[int, int, int, int, int]):
        return cls(*signal)

In [ ]:
class CompositeSignal:
    _sampling_rate: int
    _length: int
    _signal_w: NDArray[floating]
    _signal_t: NDArray[floating]
    
    def __init__(self, signal_w: NDArray[floating], signal_t: NDArray[floating]):
        self._signal_w = signal_w
        self._signal_t = signal_t
        self._find_primitives()
        
    def plot_signal(self, name: str = 'Unnamed signal') -> None:
        fig = go.Figure(layout=go.Layout(title=name, xaxis=dict(title='t-component'), yaxis=dict(title='w-component')))
        fig.add_trace(go.Scatter(x=self._signal_t, y=self._signal_w))
        fig.show()
    
    def _find_primitives(self) -> None:
        self._length = self._signal_t[-1]
        self._sampling_rate = (len(self._signal_t) - 1) / self._length
    

In [ ]:
signal = BasicSignal(1, 0.1, 0, 1000, 10)
signal2 = BasicSignal(100, 0.1, 0, 10000, 1)
signal.plot_signal('Modulating signal')
signal2.plot_signal('Carrier signal')

In [ ]:
def do_am_modulation(modulating_signal: BasicSignal, carrier_signal: BasicSignal):
    carrier = BasicSignal.copy_and_modify(carrier_signal, length=modulating_signal._length, sampling_frequency=modulating_signal._sampling_frequency)
    t_component = np.linspace(0, carrier._length, carrier._sampling_frequency * carrier._length)
    modulated_result = np.empty((1,))
    carrier_warr = carrier.get_signal_elements()[0]
    modulator_warr = modulating_signal.get_signal_elements()[0]
    for carrier_w, modulator_w in zip(carrier_warr, modulator_warr):
        modulated_result = np.append(modulated_result, (carrier_w * (1 + modulator_w/modulating_signal._amplitude)))
    return CompositeSignal(modulated_result, t_component)

In [ ]:
composite = do_am_modulation(signal, signal2)
composite.plot_signal('AM-modulated signal')

In [ ]:
INTEGRAL_SENS=10000000
def do_fm_modulation_detailed(modulating_signal: BasicSignal, carrier_signal: BasicSignal, sensitivity: int):
    carrier = BasicSignal.copy_and_modify(carrier_signal, length=modulating_signal._length, sampling_frequency=modulating_signal._sampling_frequency)
    w_mod, t_component = modulating_signal.get_signal_elements()
    
    w_res = carrier._amplitude * np.cos(2 * np.pi * carrier._frequency * t_component + 2 * np.pi * sensitivity * modulating_signal._amplitude * np.trapz(w_mod, t_component, dx=1/INTEGRAL_SENS))
    
    return CompositeSignal(w_res, t_component)

In [ ]:
def do_fm_modulation_fast(modulating_signal: BasicSignal, carrier_signal: BasicSignal, sensitivity: int) -> CompositeSignal:
    carrier = BasicSignal.copy_and_modify(carrier_signal, length=modulating_signal._length, sampling_frequency=modulating_signal._sampling_frequency)
    w_mod, t_component = modulating_signal.get_signal_elements()
    # w_res = carrier._amplitude * np.cos(2 * np.pi * (carrier._frequency + sensitivity * modulating_signal._amplitude * w_mod) * t_component)
    w_res = carrier._amplitude * np.cos(2 * np.pi * carrier._frequency * t_component + (sensitivity*modulating_signal._amplitude/modulating_signal._frequency) * np.sin(2 * np.pi * modulating_signal._frequency * t_component))
    
    return CompositeSignal(w_res, t_component)

In [ ]:
def do_fm_modulation(modulating_signal: BasicSignal, carrier_signal: BasicSignal, sensitivity: int, fast: bool = True) -> CompositeSignal:
    if fast:
        return do_fm_modulation_fast(modulating_signal, carrier_signal, sensitivity)
    else:
        return do_fm_modulation_detailed(modulating_signal, carrier_signal, sensitivity)

In [ ]:
signal = BasicSignal(1, 0.1, 0, 100000, 10)
signal2 = BasicSignal(500, 0.1, 0, 100000, 1)

fm_result = do_fm_modulation(signal, signal2, 0)
fm_result.plot_signal('FM-modulated signal')
fm_result2 = do_fm_modulation(signal, signal2, 0, False)
fm_result2.plot_signal('FM-modulated signal')

difference = fm_result._signal_w - fm_result2._signal_w
fig = go.Figure(layout=go.Layout(title='Difference Plot', xaxis=dict(title='Index'), yaxis=dict(title='Difference')))
fig.add_trace(go.Scatter(x=np.arange(len(difference)), y=difference))
fig.show()


In [ ]:
def do_pm_modulation(modulating_signal: BasicSignal, carrier_signal: BasicSignal) -> CompositeSignal:
    carrier = BasicSignal.copy_and_modify(carrier_signal, length=modulating_signal._length, sampling_frequency=modulating_signal._sampling_frequency)
    w_mod, t_component = modulating_signal.get_signal_elements()
    w_car = carrier.get_signal_elements()[0]
    w_res = np.pi/2*np.sin((w_car*t_component)+(np.pi*np.sin(w_mod*t_component)/2))
    return CompositeSignal(w_res, t_component)

In [ ]:
signal = BasicSignal(1, 0.1, 0, 10000, 1)
signal2 = BasicSignal(100, 0.1, 0, 10000, 1)
do_pm_modulation(signal, signal2).plot_signal('PM-modulated signal')